In [17]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 데이브 더 다이버 데이터 불러오기
loader = WebBaseLoader(
    web_paths=("https://namu.wiki/w/%EB%8D%B0%EC%9D%B4%EB%B8%8C%20%EB%8D%94%20%EB%8B%A4%EC%9D%B4%EB%B2%84/%EC%A7%80%EC%97%AD%20%EB%B0%8F%20%EC%83%9D%EB%AC%BC",),
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)
docs = loader.load()

In [18]:
# from pprint import pprint

# for i in docs:
#     pprint(i.page_content)

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=2000,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=100,
)

text = text_splitter.split_documents(docs)

In [20]:
# for i in text:
#     print(i.page_content)

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [21]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(text, embeddings)

In [22]:
question = "빙하통로 지역에 어떤 생물이 나와?"

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

retriever = db.as_retriever()
llm = ChatOpenAI()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True # 답변 근거 출력
)

In [24]:
result = qa_chain.invoke({"query":question})
result 

{'query': '빙하통로 지역에 어떤 생물이 나와?',
 'result': '빙하통로 지역에는 북극 망원경 물고기(Arctic Telescope Fish), 알래스카 명태(Alaska Pollock), 럼프피쉬(Lumpfish), 들창코가시장어(Snub-nosed Spiny Eel) 등의 생물이 등장합니다.',
 'source_documents': [Document(id='282089ba-5415-4828-bc28-fbf88183f9f3', metadata={'source': 'https://namu.wiki/w/%EB%8D%B0%EC%9D%B4%EB%B8%8C%20%EB%8D%94%20%EB%8B%A4%EC%9D%B4%EB%B2%84/%EC%A7%80%EC%97%AD%20%EB%B0%8F%20%EC%83%9D%EB%AC%BC', 'title': '데이브 더 다이버/지역 및 생물 - 나무위키', 'language': 'ko'}, page_content='빙하 지역에 있는 비악상어/그린란드 상어가 더 비싸게 받는데다가 열수 지역이 열리면 둔클레오스테우스를 잡고 강화해도 참다랑어보다 더 많이 벌게 된다.맵에 스폰된다면 다랑어 무리가 왕복하는 구간과는 데이브와 화면 하나 정도 크기를 두고 무언가 물에 빠지는 듯한 요란한 물소리가 나는데 이걸로 미리 다랑어 무리가 근처에 스폰되었는가 쉽게 판별할 수 있다.갈무리로 살점을 얻는 대형 어종인데다 3부위가 동시에 나오기 때문에 한 마리당 얻을 수 있는 살점이 많아서 붉은살/중뱃살은 강화 난이도도 쉬운 편이다. 게다가 참치회가 현실에서도 고급 요리인 점을 반영했는지 디폴트 상태의 초밥 가격의 상당히 높다. 안정적으로 획득할 수 있는 순간부터 수입을 크게 상승시켜주지만 스폰 여부는 랜덤이라서 은근 운이 따라줘야 한다.패치 이전 마구로동은 계란과 쌀 등 부수적인 재료를 많이 요구하지만 그래도 가장 무난한 참치 파티 대책으로 평 받았는데, 2023년 10월 11일 패치로 핫페퍼튜나 같은 다른 요리들이 더 추가되면서 다양성이 늘